# Harry's House - Streaming Api 20/05


In [19]:
#per emotional analysis
!pip install feel-it

In [85]:
#per emotional analysis
!pip install text2emotion

  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171047 sha256=f55d395f648fad11a7f8cecde4cb827132cbc3ec4993cde0c7b8fbf7a63d947a
  Stored in directory: c:\users\sarac\appdata\local\pip\cache\wheels\fa\7a\e9\22dd0515e1bad255e51663ee513a2fa839c95934c5fc301090
Successfully built emoji


In [37]:
#per trasformare output del modello in risultati probabilistici
!pip install scipy 

In [2]:
#per nlp sentiment analysis
!pip install transformers

In [4]:
#IMPORT

import tweepy
import CONFIG
import csv
import os
import json
import numpy as np
import pandas as pd
import time
import re
import text2emotion as te

from collections import Counter
from feel_it import EmotionClassifier
from feel_it import SentimentClassifier

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

File in cui abbiamo salavto tutti i tweet raccolti dalle 6 alle 9 di mattina del 20 Maggio

In [5]:
streaming_api = json.load(open('streaming_tweet.json'))

In [6]:
len(streaming_api)

166511

Elimino tutti i tweet retweettati

In [7]:
count_not_RT = len(streaming_api)
streaming_tweets = []

for t in streaming_api:
    tweet = t['data']
    
    if tweet['text'][0] == "R" and tweet['text'][1] == "T":
        #print("RT")
        count_not_RT -= 1
    else:
        streaming_tweets.append(t)

print(count_not_RT)
print(len(streaming_tweets))

66961
66961


Lista di tutti i possibili **UTENTI** (non solo gli autori, ma anche di loro che sono stati taggati etc.)

In [8]:
users = {}

for t in streaming_tweets:
    users_list = t['includes']
    for u in users_list['users']:
        users[u['id']] = u

In [9]:
print(len(users))

34833


In [10]:
streaming_tweets[1]['data']

{'author_id': '1555409701',
 'created_at': '2022-05-20T04:03:10.000Z',
 'entities': {'hashtags': [{'start': 77, 'end': 89, 'tag': 'HarrysHouse'}],
  'mentions': [{'start': 0,
    'end': 13,
    'username': 'Harry_Styles',
    'id': '181561712'}],
  'urls': [{'start': 90,
    'end': 113,
    'url': 'https://t.co/ZE9gBvjzT0',
    'expanded_url': 'https://twitter.com/satelIiterryy/status/1527500151707009024/photo/1',
    'display_url': 'pic.twitter.com/ZE9gBvjzT0',
    'media_key': '3_1527500145394589697'}]},
 'geo': {},
 'id': '1527500151707009024',
 'lang': 'en',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0},
 'text': '@Harry_Styles cried my eyes out, this song is so personal to me. thank you \U0001faf6🏼#HarrysHouse https://t.co/ZE9gBvjzT0'}

In [11]:
streaming_tweets[1]['includes']['users']

[{'id': '1555409701',
  'location': 'she/her | 20 | aus',
  'name': 'angie HH SPOILERS',
  'public_metrics': {'followers_count': 2018,
   'following_count': 295,
   'tweet_count': 55312,
   'listed_count': 99},
  'username': 'satelIiterryy',
  'verified': False},
 {'id': '181561712',
  'name': 'Harry Styles.',
  'public_metrics': {'followers_count': 37729658,
   'following_count': 1814,
   'tweet_count': 5619,
   'listed_count': 143462},
  'username': 'Harry_Styles',
  'verified': True}]

Lista di tutti gli **AUTORI**

In [12]:
authors = {}

for t in streaming_tweets:
    id_author = t['data']['author_id']
    
    #cerchiamo l'id in users
    author = users[id_author]
    
    authors[id_author] = author

In [13]:
print(len(authors))

33938


Filtriamo le informazioni più importanti per effettuare la **Sentiment Analysis**. Ciò che ci serve è:
- data di creazione
- testo

In [14]:
tweets_filtered = []

for t in streaming_tweets:
    created_at = t['data']['created_at']
    text = t['data']['text']
    author_id = t['data']['author_id']
    lang = t['data']['lang']
    
    
    #togliamo url e mentions
    text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text)
    text = re.sub("#[A-Za-z0-9_]+","", text)
    #sistemiamo il testo
    text = text.replace('\n', ' ')
    #togliamo i multiple spaces
    text = re.sub(' +', ' ', text)
    text = text.strip()
    
    dict_obj = {
        'author_id': author_id,
        'created_at': created_at,
        'text': text,
        'lang': lang
    }
    
    tweets_filtered.append(dict_obj)

In [18]:
tweets_filtered[1]

{'author_id': '1555409701',
 'created_at': '2022-05-20T04:03:10.000Z',
 'text': 'cried my eyes out, this song is so personal to me. thank you \U0001faf6🏼',
 'lang': 'en'}

In [16]:
path = os.getcwd() + '\HARRY_HOUSE_DATA'
os.chdir(path)
print(os.getcwd())

C:\Users\sarac\OneDrive - Università degli Studi di Milano\UNI 2022\_SOCIALMEDIA\#harryhouse\HARRY_HOUSE_DATA


## Emotional Analysis

In [14]:
#oggetto per emotional analysis in ita
emotion_clf = EmotionClassifier()

In [15]:
def analyze_emotion(emotion_dict):
    max_value = max(emotion_dict, key=emotion_dict.get)
    
    if max_value  == 'Happy':
        return "joy"
    elif max_value  == 'Angry':
        return 'anger'
    elif max_value  == 'Surprise':
        return 'surpise'
    elif max_value  == 'Sad':
        return 'sadness'
    else:
        return 'fear'

In [21]:
list_emotions = []
count = 0

In [28]:
emotion = ''

for t in tweets_filtered:
    text = []
    if t['lang'] == 'it':
        text.append(t['text'])
        emotion = emotion_clf.predict(text)[0]
    elif t['lang'] == 'en':
        emotion = analyze_emotion(te.get_emotion(t['text']))
    else:
        print("continue")
        continue
    
    obj = [t['created_at'], t['text'], emotion]
    list_emotions.append(obj)

count:  60001
count:  60002
count:  60003
count:  60004
count:  60005
count:  60006
continue
count:  60007
continue
count:  60008
count:  60009
continue
count:  60010
continue
count:  60011
continue
count:  60012
count:  60013
count:  60014
continue
count:  60015
continue
count:  60016
count:  60017
count:  60018
continue
count:  60019
continue
count:  60020
count:  60021
count:  60022
continue
count:  60023
count:  60024
continue
count:  60025
count:  60026
count:  60027
continue
count:  60028
count:  60029
continue
count:  60030
continue
count:  60031
continue
count:  60032
count:  60033
continue
count:  60034
count:  60035
count:  60036
count:  60037
continue
count:  60038
continue
count:  60039
count:  60040
count:  60041
continue
count:  60042
count:  60043
count:  60044
count:  60045
continue
count:  60046
continue
count:  60047
continue
count:  60048
continue
count:  60049
count:  60050
continue
count:  60051
continue
count:  60052
count:  60053
count:  60054
count:  60055
count

count:  60475
count:  60476
count:  60477
count:  60478
count:  60479
continue
count:  60480
count:  60481
continue
count:  60482
continue
count:  60483
count:  60484
count:  60485
count:  60486
count:  60487
count:  60488
continue
count:  60489
count:  60490
continue
count:  60491
continue
count:  60492
count:  60493
count:  60494
count:  60495
count:  60496
count:  60497
continue
count:  60498
count:  60499
continue
count:  60500
continue
count:  60501
count:  60502
count:  60503
count:  60504
count:  60505
continue
count:  60506
count:  60507
count:  60508
count:  60509
continue
count:  60510
count:  60511
count:  60512
count:  60513
continue
count:  60514
continue
count:  60515
count:  60516
continue
count:  60517
continue
count:  60518
count:  60519
count:  60520
count:  60521
continue
count:  60522
count:  60523
continue
count:  60524
count:  60525
count:  60526
count:  60527
count:  60528
continue
count:  60529
continue
count:  60530
count:  60531
count:  60532
count:  60533
cou

count:  60957
continue
count:  60958
count:  60959
count:  60960
continue
count:  60961
count:  60962
continue
count:  60963
continue
count:  60964
count:  60965
continue
count:  60966
count:  60967
count:  60968
continue
count:  60969
count:  60970
count:  60971
count:  60972
count:  60973
count:  60974
count:  60975
count:  60976
continue
count:  60977
count:  60978
count:  60979
count:  60980
count:  60981
count:  60982
count:  60983
count:  60984
count:  60985
count:  60986
continue
count:  60987
count:  60988
continue
count:  60989
count:  60990
count:  60991
continue
count:  60992
count:  60993
count:  60994
count:  60995
count:  60996
continue
count:  60997
count:  60998
count:  60999
count:  61000
count:  61001
count:  61002
count:  61003
count:  61004
count:  61005
continue
count:  61006
continue
count:  61007
count:  61008
count:  61009
continue
count:  61010
continue
count:  61011
count:  61012
count:  61013
count:  61014
count:  61015
continue
count:  61016
count:  61017
co

count:  61446
continue
count:  61447
continue
count:  61448
continue
count:  61449
count:  61450
count:  61451
count:  61452
count:  61453
count:  61454
count:  61455
count:  61456
count:  61457
continue
count:  61458
continue
count:  61459
count:  61460
continue
count:  61461
count:  61462
continue
count:  61463
count:  61464
count:  61465
count:  61466
count:  61467
count:  61468
count:  61469
count:  61470
count:  61471
count:  61472
count:  61473
continue
count:  61474
count:  61475
count:  61476
continue
count:  61477
count:  61478
count:  61479
count:  61480
continue
count:  61481
continue
count:  61482
continue
count:  61483
count:  61484
count:  61485
count:  61486
count:  61487
count:  61488
count:  61489
continue
count:  61490
count:  61491
count:  61492
count:  61493
count:  61494
count:  61495
count:  61496
count:  61497
count:  61498
count:  61499
count:  61500
count:  61501
count:  61502
continue
count:  61503
count:  61504
count:  61505
count:  61506
count:  61507
count:

count:  61937
count:  61938
continue
count:  61939
count:  61940
count:  61941
continue
count:  61942
continue
count:  61943
count:  61944
count:  61945
count:  61946
count:  61947
continue
count:  61948
count:  61949
count:  61950
continue
count:  61951
count:  61952
count:  61953
continue
count:  61954
continue
count:  61955
continue
count:  61956
count:  61957
continue
count:  61958
continue
count:  61959
count:  61960
continue
count:  61961
count:  61962
count:  61963
count:  61964
count:  61965
count:  61966
count:  61967
count:  61968
continue
count:  61969
count:  61970
continue
count:  61971
count:  61972
count:  61973
count:  61974
continue
count:  61975
count:  61976
count:  61977
count:  61978
continue
count:  61979
count:  61980
continue
count:  61981
count:  61982
continue
count:  61983
count:  61984
count:  61985
count:  61986
count:  61987
count:  61988
continue
count:  61989
continue
count:  61990
count:  61991
count:  61992
count:  61993
count:  61994
continue
count:  

...

count:  64834
continue
count:  64835
continue
count:  64836
count:  64837
count:  64838
count:  64839
count:  64840
count:  64841
count:  64842
count:  64843
count:  64844
continue
count:  64845
count:  64846
count:  64847
count:  64848
count:  64849
count:  64850
count:  64851
continue
count:  64852
count:  64853
count:  64854
continue
count:  64855
count:  64856
continue
count:  64857
count:  64858
continue
count:  64859
count:  64860
count:  64861
continue
count:  64862
continue
count:  64863
count:  64864
continue
count:  64865
continue
count:  64866
continue
count:  64867
count:  64868
count:  64869
continue
count:  64870
continue
count:  64871
count:  64872
count:  64873
count:  64874
count:  64875
count:  64876
continue
count:  64877
count:  64878
continue
count:  64879
continue
count:  64880
continue
count:  64881
continue
count:  64882
count:  64883
count:  64884
continue
count:  64885
continue
count:  64886
count:  64887
count:  64888
count:  64889
continue
count:  64890
coun

count:  65308
continue
count:  65309
continue
count:  65310
count:  65311
continue
count:  65312
count:  65313
count:  65314
count:  65315
count:  65316
count:  65317
count:  65318
continue
count:  65319
count:  65320
count:  65321
count:  65322
count:  65323
count:  65324
continue
count:  65325
continue
count:  65326
count:  65327
continue
count:  65328
continue
count:  65329
count:  65330
count:  65331
count:  65332
count:  65333
count:  65334
count:  65335
count:  65336
count:  65337
count:  65338
count:  65339
continue
count:  65340
count:  65341
continue
count:  65342
count:  65343
count:  65344
count:  65345
continue
count:  65346
continue
count:  65347
count:  65348
continue
count:  65349
count:  65350
continue
count:  65351
continue
count:  65352
continue
count:  65353
continue
count:  65354
count:  65355
continue
count:  65356
count:  65357
count:  65358
count:  65359
count:  65360
count:  65361
count:  65362
count:  65363
continue
count:  65364
continue
count:  65365
count:  

count:  65780
count:  65781
continue
count:  65782
continue
count:  65783
continue
count:  65784
count:  65785
count:  65786
count:  65787
continue
count:  65788
continue
count:  65789
continue
count:  65790
count:  65791
count:  65792
continue
count:  65793
count:  65794
count:  65795
count:  65796
count:  65797
continue
count:  65798
continue
count:  65799
count:  65800
continue
count:  65801
count:  65802
continue
count:  65803
count:  65804
count:  65805
count:  65806
count:  65807
count:  65808
count:  65809
count:  65810
continue
count:  65811
continue
count:  65812
continue
count:  65813
count:  65814
count:  65815
count:  65816
count:  65817
continue
count:  65818
continue
count:  65819
count:  65820
continue
count:  65821
count:  65822
count:  65823
count:  65824
count:  65825
continue
count:  65826
count:  65827
count:  65828
count:  65829
count:  65830
count:  65831
count:  65832
count:  65833
continue
count:  65834
continue
count:  65835
count:  65836
count:  65837
continue

count:  66252
count:  66253
continue
count:  66254
count:  66255
continue
count:  66256
count:  66257
continue
count:  66258
continue
count:  66259
count:  66260
count:  66261
continue
count:  66262
continue
count:  66263
continue
count:  66264
count:  66265
count:  66266
count:  66267
count:  66268
continue
count:  66269
continue
count:  66270
continue
count:  66271
continue
count:  66272
continue
count:  66273
continue
count:  66274
continue
count:  66275
count:  66276
count:  66277
continue
count:  66278
continue
count:  66279
count:  66280
count:  66281
count:  66282
continue
count:  66283
continue
count:  66284
count:  66285
continue
count:  66286
count:  66287
continue
count:  66288
continue
count:  66289
count:  66290
continue
count:  66291
continue
count:  66292
count:  66293
continue
count:  66294
continue
count:  66295
count:  66296
count:  66297
count:  66298
count:  66299
continue
count:  66300
count:  66301
continue
count:  66302
continue
count:  66303
count:  66304
count:

count:  66721
count:  66722
continue
count:  66723
continue
count:  66724
continue
count:  66725
continue
count:  66726
count:  66727
count:  66728
count:  66729
count:  66730
continue
count:  66731
count:  66732
continue
count:  66733
continue
count:  66734
count:  66735
count:  66736
count:  66737
continue
count:  66738
continue
count:  66739
count:  66740
continue
count:  66741
count:  66742
count:  66743
count:  66744
continue
count:  66745
continue
count:  66746
continue
count:  66747
count:  66748
continue
count:  66749
count:  66750
count:  66751
count:  66752
continue
count:  66753
continue
count:  66754
count:  66755
continue
count:  66756
continue
count:  66757
count:  66758
continue
count:  66759
count:  66760
count:  66761
count:  66762
continue
count:  66763
count:  66764
continue
count:  66765
count:  66766
count:  66767
count:  66768
count:  66769
continue
count:  66770
count:  66771
count:  66772
count:  66773
count:  66774
count:  66775
continue
count:  66776
continue


In [29]:
len(list_emotions)

36768

In [31]:
print(list_emotions[0])

['2022-05-20T04:03:10.000Z', 'cried my eyes out, this song is so personal to me. thank you \U0001faf6🏼', 'fear']


In [34]:
header = ['created_at', 'text', 'emotion']

with open('emotion_streaming_api.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(list_emotions)

## Sentiment Analysis

In [50]:
sentiment_clf = SentimentClassifier()

In [51]:
#download model roBerta and tokenizer
roberta = 'cardiffnlp/twitter-roberta-base-sentiment'
model = AutoModelForSequenceClassification.from_pretrained(roberta)

tokenizer = AutoTokenizer.from_pretrained(roberta)

In [57]:
# label di output della classificazione 
lables = ['Negative', 'Neutral', 'Positive']

In [58]:
# sentiment analysis
list_sentiment = []
count = 0

In [59]:
def analyze_sentiment(scores):
    scores_list = scores.tolist()
    max_score = max(scores_list)
    i = scores_list.index(max_score)
    return lables[i].lower()

In [ ]:
sentiment = ''

for t in tweets_filtered:
    tweet = t['text']
    count += 1
    if t['lang'] == 'en':

        #trasformazione del testi in vect
        encoded_tweet = tokenizer(tweet, return_tensors='pt')  

        #diamo al modello il tweet
        output = model(**encoded_tweet)

        #trasformiamo l'output in numeri per capire la % di sentimento
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        print(count)
        sentiment = analyze_sentiment(scores)

    elif t['lang'] == 'it':
        tweet_list = []
        tweet_list.append(text)
        sentiment = sentiment_clf.predict(tweet_list)[0]
    else:
        print(count)
        continue
    
    obj = [[t['created_at'], tweet, sentiment]]
    list_sentiment.append(obj)


In [61]:
len(list_sentiment)

36768

In [62]:
list_sentiment[0]

[['2022-05-20T04:03:10.000Z',
  'cried my eyes out, this song is so personal to me. thank you \U0001faf6🏼',
  'positive']]

In [63]:
header = ['created_at', 'text', 'sentiment']

with open('sentiment_streaming_api.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(list_sentiment)